In [61]:
from pathlib import Path
import subprocess
import sys
import json
from pprint import pprint

p = Path()
ctags_exe_path = list(p.glob("./ctags.exe"))

source_dir_path = Path().absolute()
source_dir_path = source_dir_path.parent
source_dir_path = source_dir_path/"source"


def exit_system():
    print("calling sys.exit()")
    sys.exit()

if len(ctags_exe_path) == 0:
    raise Exception("error:", "ctags.exe was not found.")

dir_command = "dir /B /S ..\\*.c > list.txt"
res = subprocess.run(dir_command, shell=True, capture_output=True)
if res.returncode != 0:
    raise Exception()


# ctags -R --language-force=C --kinds-C=* --fields=* --output-format=json
ctags_command = []
ctags_command.append("-R")
ctags_command.append("--language-force=C")
ctags_command.append("--kinds-C=*")
ctags_command.append("--fields=*")
ctags_command.append("--output-format=json")
ctags_command.append(" > tags.json")

ctags_command = "ctags -L list.txt --language-force=C --kinds-C=* --fields=* --output-format=json > tags.json"

res = subprocess.run(ctags_command, shell=True, capture_output=True)
if res.returncode != 0:
    raise Exception()



with open("tags.json", "r") as f:
    lines = f.read().split("\n")

tags = {}

for line in lines:
    if line.startswith("!") or not line:
        continue
    tag = json.loads(line)
    
    if "name" in tag:
        tag_name = tag["name"]
    else:
        print("warning:", "no tag name in line.")
        continue
        
    if tag_name not in tags:
        tags[tag_name] = []

    tags[tag_name].append(tag)



class CHECKER:
    def __init__(self, name=None):
        self.name = name
        self.externs = []
        self.publics = []
        self.functions = []
        self.prototypes = []
        self.functions_defined_in = []
        self.functions_externed_in = []
        self.errors = []
        self.var_or_func = ""
        self.checks = []
        
    
    def check_all(self):
        self.find_function_defined_in()
        self.check_1()
        self.check_2()
        self.check_3()
        self.check_4()
        self.check_5()

    def find_function_defined_in(self):
        c_functions = {}
        c_prototypes = {}
        for tag in self.functions:
            c_functions[tag["path"]] = True
        for tag in self.prototypes:
            c_prototypes[tag["path"]] = True

        self.function_defined_in = []
        self.function_externed_in = []
        for c_prototype in c_prototypes:
            if c_prototype not in c_functions:
                self.function_externed_in.append(c_prototype)
        for c_function in c_functions:
            self.function_defined_in.append(c_function)


    # Check undefined variable
    def check_1(self):
        if self.var_or_func != "var":
            return 0
        if len(self.externs) >= 1 and len(self.publics) == 0:
            for tag in self.externs:
                error = "Error: Undefined variable", self.name, "in", Path(tag["path"]).relative_to(source_dir_path)
                #print(error)
                self.errors.append(error)
                return 1
        else:
            return 0

    # Check dupliacte variable
    def check_2(self):
        if self.var_or_func != "var":
            return 0
        if len(self.publics) >= 2:
            for tag in self.publics:
                error = "Error: Duplicate variable", self.name, "in", Path(tag["path"]).relative_to(source_dir_path)
                #print(error)
                self.errors.append(error)
                return 1
        else:
            return 0

    # Check undefined function
    def check_3(self):
        if self.var_or_func != "func":
            return 0
        if len(self.function_externed_in) >= 1 and len(self.function_defined_in) == 0:
            for cpath in self.function_externed_in:
                error = "Error: Undefined function", self.name, "in", Path(cpath).relative_to(source_dir_path)
                #print(error)
                self.errors.append(error)
                return 1
        else:
            return 0
    
    # Check dupliacte function
    def check_4(self):
        if self.var_or_func != "func":
            return 0
        if len(self.function_defined_in) >= 2:
            for cpath in self.function_defined_in:
                error = "Error: Duplicate function", self.name, "in", Path(cpath).relative_to(source_dir_path)
                #print(error)
                self.errors.append(error)
                return 1
        else:
            return 0
    
    # Check Nonexterned function
    def check_5(self):
        if self.var_or_func != "func":
            return 0
        if len(self.function_externed_in) == 0 and len(self.function_defined_in) >= 1:
            for cpath in self.function_defined_in:
                error = "Warning: Nonexterned function", self.name, "in", Path(cpath).relative_to(source_dir_path)
                #print(error)
                self.errors.append(error)
                return 1
        else:
            return 0
    
# link error check
for tag_name in tags:
    tag_list = tags[tag_name]
    
    for tag in tag_list:
        check = CHECKER(tag_name)
        if tag["kind"] == "externvar":
            check.externs.append(tag)
        elif tag["kind"] == "variable":
            check.publics.append(tag)
        elif tag["kind"] == "function":
            check.functions.append(tag)
        elif tag["kind"] == "prototype":
            check.prototypes.append(tag)
        
        if check.externs or check.publics:
            check.var_or_func = "var"
        elif check.functions or check.prototypes:
            check.var_or_func = "func"
        else:
            continue
        
        
        if check.var_or_func == "func":
            for tag in check.functions:
                pass

    check.check_all()
    print(check.name)
    pprint(check.errors)


            







a
[]
asfasdfsadf
[('Error: Undefined function', 'asfasdfsadf', 'in', WindowsPath('a.c'))]
b
[('Error: Undefined variable', 'b', 'in', WindowsPath('b.c'))]
func1
[('Warning: Nonused function', 'func1', 'in', WindowsPath('a.c'))]
func1a
[('Error: Undefined function', 'func1a', 'in', WindowsPath('b.c'))]
func2
[('Warning: Nonused function', 'func2', 'in', WindowsPath('b.c'))]
hoge
[('Error: Undefined variable', 'hoge', 'in', WindowsPath('asdf/c.c'))]


In [48]:

source_dir_path = Path().absolute()
source_dir_path = source_dir_path.parent
source_dir_path = source_dir_path/"source"
